# Objective of the competition

1. The objective of this competition is to create a machine learning model to classify fields by crop type from images collected during the growing season by the Sentinel-2 satellite. 
2. The fields pictured in this training set are across western Kenya, and the images were collected by the PlantVillage team.

In [ ]:
!pip install tifffile
!pip install tqdm

In [ ]:
# Required libraries
import requests
from urllib.parse import urlparse
from pathlib import Path
import datetime
import tifffile as tiff

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from glob import glob
from io import BytesIO
import numpy as np
import pandas as pd

from skimage import exposure

%matplotlib inline

# Step 1: Datasets

1. Load datasets from their folders.
2. Analyse the data to get a better understanding of what you are working with.
3. Plot data if there is any need.

In [ ]:
data_path = '/kaggle/input/crop-type-kenya/ref_african_crops_kenya_02_source/ref_african_crops_kenya_02_source/'
labels_path = '../input/kenya-labels/ref_african_crops_kenya_02_labels/'
!ls {labels_path}

In [ ]:
sample_submission = pd.read_csv('../input/sample-sub/SampleSubmission (3).csv')

In [ ]:
pd.DataFrame(sample_submission)

In [ ]:
def load_file(img_path):
    """Takes a path to the download archive and the path within the archive to the image and returns a numpy array."""
    
    return tiff.imread(str(img_path))

#### Get a list of dates that an observation from Sentinel-2 is provided for from the currently downloaded imagery


In [ ]:

tile_dates = dict()

def filter_tifs(name):
    return name.endswith('.tif')

for f in glob(str(data_path + '**/*.tif')):
    tif_path = Path(f)
    _, tile_id, tile_date = tif_path.parent.name.rsplit('_', 2)

    if tile_id not in tile_dates:
        tile_dates[tile_id] = set()

    tile_dates[tile_id].add(tile_date)

In [ ]:
for tile in sorted(tile_dates.keys()):
    print(f'Tile ID: {tile}')
    dates = sorted(list(tile_dates[tile]))
    print(f'Dates: {", ".join(dates)}')
    print('')

In [ ]:
bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'CLD']

### Sample file to load:

Select image from tile **02 dated 20190825 on band 3.**

In [ ]:
# Sample file to load:
tile = '02'
date_ = '20190825'
band = 'B03'

img_path = data_path + f'ref_african_crops_kenya_02_tile_{tile}_{date_}/{band}.tif'
band_data = load_file(img_path)

In [ ]:
#Plot Band
fig = plt.figure(figsize=(7,7))
plt.imshow(band_data)

#### Load images from the datasets

In [ ]:
selected_tile = list(tile_dates.keys())[0]
dates = sorted(tile_dates[selected_tile])

bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'CLD']
def load_image(date):
    img = list()
    for band in bands:
        img_path = data_path + f"ref_african_crops_kenya_02_tile_{selected_tile}_{date}/{band}.tif"
        img.append(load_file(img_path))
    return np.dstack(img)

def load_timeseries():
    tstack = list()
    with tqdm(dates, total=len(dates), desc="reading images") as pbar:
        for date in pbar:
            tstack.append(load_image(date))
    return np.stack(tstack) 

timeseries = load_timeseries()
print(timeseries.shape)

# Step 2: Features

1. RGB
2. False color
3. NDVI

The features will help with getting a clear picture of where there is vegetation concentration

In [ ]:
# List of Sentinel-2 bands in the dataset

rgb = np.array(['B04','B03', 'B02'])
rgb_idx = [bands.index(b) for b in rgb]
rgb_stack = timeseries[:,:,:,rgb_idx]

false_color = np.array(['B08','B04', 'B03'])
false_color_idx = [bands.index(b) for b in false_color]
false_color_stack = timeseries[:,:,:,false_color_idx]

perc_rgb = np.percentile(rgb_stack.mean(2), (10, 90))
perc_false_color = np.percentile(np.median(false_color_stack,axis=2), (2, 98))

red = timeseries[:,:,:,bands.index("B04")]
nir = timeseries[:,:,:,bands.index("B08")]
ndvi = (red-nir) / (red+nir)

fig,axs = plt.subplots(len(dates),3,figsize=(15,len(dates)*8))

with tqdm(range(len(dates)),total=len(dates)) as pbar:
    for date_idx in pbar:
        
        
        ax = axs[date_idx,0]
        rgb_img = rgb_stack[date_idx]
        rgb_img = exposure.rescale_intensity(rgb_img, in_range=(perc_rgb[0],perc_rgb[1]))
        ax.imshow(rgb_img)
        ax.axis('off')
        dt = dates[date_idx]
        ax.set_title(f"RGB {dt} {rgb}")

        ax = axs[date_idx,1]
        false_color_img = false_color_stack[date_idx]
        false_color_img = exposure.rescale_intensity(false_color_img, in_range=(perc_false_color[0],perc_false_color[1]))
        ax.imshow(false_color_img)
        ax.axis('off')
        dt = dates[date_idx]
        ax.set_title(f"False-Color {dt} {false_color}")

        ax = axs[date_idx,2]
        ndvi_img = ndvi[date_idx]
        ax.imshow(ndvi_img, cmap="Greens_r")
        ax.axis('off')
        dt = dates[date_idx]
        ax.set_title(f"NDVI {dt} (B08-B04)/(B08+B04)")

fig.tight_layout()

### Getting data for each pixel in fields

1. Read in the labels, find the pixel locations of all the fields (most are only a few pixels in size) and store the pixel locations, the field ID and the label.

2. loop through all the images, sampling the different image bands to get the values for each pixel in each field. 

   **Convert labels and field ids to data frames**

In [ ]:
row_locs = []
col_locs = []
field_ids = []
labels = []
tiles = []
for tile in range(4):
  field_id = labels_path + f'ref_african_crops_kenya_02_tile_{tile}_label/field_ids.tif'
  label = labels_path + f'ref_african_crops_kenya_02_tile_{tile}_label/labels.tif'
  field_arr = load_file(field_id)
  label_arr = load_file(label)
  for row in range(len(field_arr)):
    for col in range(len(field_arr[0])):
      if field_arr[row][col] != 0:
        row_locs.append(row)
        col_locs.append(col)
        field_ids.append(field_arr[row][col])
        labels.append(label_arr[row][col])
        tiles.append(tile)

labels_field_id_df = pd.DataFrame({
    'fid':field_ids,
    'label':labels,
    'row_loc': row_locs,
    'col_loc':col_locs,
    'tile':tiles
})

labels_field_id_df.groupby('fid').count().shape

pd.DataFrame(labels_field_id_df)


In [ ]:
# List of dates that an observation from Sentinel-2 is provided in the training dataset
dates = [datetime.datetime(2019, 6, 6, 8, 10, 7),
         datetime.datetime(2019, 7, 1, 8, 10, 4),
         datetime.datetime(2019, 7, 6, 8, 10, 8),
         datetime.datetime(2019, 7, 11, 8, 10, 4),
         datetime.datetime(2019, 7, 21, 8, 10, 4),
         datetime.datetime(2019, 8, 5, 8, 10, 7),
         datetime.datetime(2019, 8, 15, 8, 10, 6),
         datetime.datetime(2019, 8, 25, 8, 10, 4),
         datetime.datetime(2019, 9, 9, 8, 9, 58),
         datetime.datetime(2019, 9, 19, 8, 9, 59),
         datetime.datetime(2019, 9, 24, 8, 9, 59),
         datetime.datetime(2019, 10, 4, 8, 10),
         datetime.datetime(2019, 11, 3, 8, 10)]

In [ ]:
# Sample the bands at different dates as columns in our new dataframe
col_names = []
col_values = []

for tile in range(4): # 1) For each tile
  print('Tile: ', tile)
  for d in dates: # 2) For each date
    print(str(d))
    d = ''.join(str(d.date()).split('-')) # Nice date string
    t = '0'+str(tile)
    for b in bands: # 3) For each band
      col_name = d+'_'+b
      
      if tile == 0:
        # If the column doesn't exist, create it and populate with 0s
        labels_field_id_df[col_name] = 0

      # Load bands
    
      im = load_file(img_path)
      
      # Going four levels deep. Each second on the outside is four weeks in this loop
      # If we die here, there's no waking up.....
      vals = []
      for row, col in labels_field_id_df.loc[labels_field_id_df.tile == tile][['row_loc', 'col_loc']].values: # 4) For each location of a pixel in a field
        vals.append(im[row][col])
      labels_field_id_df.loc[labels_field_id_df.tile == tile, col_name] = vals
labels_field_id_df.head()


In [ ]:
#Save created dataframe
labels_field_id_df.to_csv('spatial_data.csv', index=False)

In [ ]:
spatial_data = pd.read_csv('spatial_data.csv')

spatial_data.head()

In [ ]:
spatial_data.shape

In [ ]:
#Merge spatial_data with labels_field_ids_df later

row_data = spatial_data.groupby('fid')['row_loc'].nunique()
column_data = spatial_data.groupby('fid')['col_loc'].nunique()

pixels_data = spatial_data.groupby('fid')['label'].nunique()


pd.DataFrame(pixels_data)

In [ ]:
#Grouped Data
spatial_data_grouped = spatial_data.groupby('fid', as_index=False).mean()

pd.DataFrame(spatial_data_grouped)

# Step 3: Model

### 1st modelling
Features used:

1. Pixel values of each of the 12 bands, INCLUDING the cloud probabilities (for the 13 timestamps)
2. Vegetation/Spectral indices like NDVI, GNDVI, AVI etc, and relevant statistics related to the indices like mean, max etc.
3. Spatial features - row_size, column_size (both indicating height, and width), area of field, and number of pixels covered by a field in the area computed.
4. Named **model_one_**


### 2nd modelling (Pixel related)
Features used:

1. Pixel values of each of the 12 bands, EXCLUDING the cloud probabilities (for the 13 timestamps)
2. Statistics related to pixel values of each band like mean, max etc
3. Named **model_two_**

In [ ]:
cloud_columns = ['20190606_CLD', '20190701_CLD', '20190706_CLD', '20190711_CLD', '20190721_CLD', '20190805_CLD', '20190815_CLD', '20190825_CLD', '20190909_CLD', '20190919_CLD', '20190924_CLD', '20191004_CLD', '20191103_CLD']


In [ ]:
#Separate dataframes for 1st and 2nd models

model_one = spatial_data_grouped.copy()

model_two = spatial_data_grouped.copy()

#Drop the cloud columns

model_two.drop(columns = cloud_columns + ['fid'], inplace = True)

pd.DataFrame(model_two)

In [ ]:

#Drop columns not required for training

model_one_train = model_one.drop(columns = ['row_loc', 'col_loc', 'tile'])
model_two_train = model_two.drop(columns = ['row_loc', 'col_loc','tile'])

pd.DataFrame(model_two_train)

### Vegetation/Spectral indices
1. Normalized Difference Vegetation Index (NDVI)
2. Green Normalized Difference Vegetation Index (GNDVI)
3. Enhanced Vegetation Index (EVI)
4. Enhanced Vegetation Index 2 (EVI2)
5. Advanced Vegetation Index (AVI)
6. Bare Soil Index (BSI)
7. Shadow Index (SI)
8. Normalized Difference Water Index (NDWI)
9. Normalized Difference Moisture Index (NDMI)
10. Normalized Pigment Chlorophyll Ratio Index (NPCRI)

In [ ]:
spectral_indices = ["NDVI", "GNDVI", "EVI", "EVI2", "AVI", "BSI", "SI", "NDWI", "NDMI", "NPCRI"]


In [ ]:
for i in range(13):
#     Band Pixel values per timestamp
    band_1 = model_one_train.filter(like = "B01").values[:,i]
    band_2 = model_one_train.filter(like = "B02").values[:,i]
    band_3 = model_one_train.filter(like = "B03").values[:,i]
    band_4 = model_one_train.filter(like = "B04").values[:,i]
    band_5 = model_one_train.filter(like = "B05").values[:,i]
    band_6 = model_one_train.filter(like = "B06").values[:,i]
    band_7 = model_one_train.filter(like = "B07").values[:,i]
    band_8 = model_one_train.filter(like = "B08").values[:,i]
    band_9 = model_one_train.filter(like = "B09").values[:,i]    
    band_11 = model_one_train.filter(like = "B11").values[:,i]
    band_12 = model_one_train.filter(like = "B12").values[:,i]
    
#     Computation of spectral indices
    ndvi = (band_8 - band_4) / (band_8 + band_4)
    gndvi = (band_8 - band_3) / (band_8 + band_3)
    evi = 2.5 * (band_8 - band_4) / ((band_8 + 6.0 * band_4 - 7.5 * band_2) + 1.0)    
    evi2 = 2.4 * (band_8 - band_4) / (band_8 + band_4 + 1.0)
    avi = (band_8 * (1 - band_4) * (band_8 - band_4))
    bsi = ((band_11 + band_4) - (band_8 + band_2)) / ((band_11 + band_4) + (band_8 + band_2))
    si = ((1 - band_2) * (1 - band_3) * (1 - band_4))
    ndwi = (band_3 - band_8) / (band_3 + band_8)
    ndmi = (band_8 - band_11) / (band_8 + band_11)
    npcri = (band_4 - band_2) / (band_4 + band_2) 
    
#     Add spectral indices as features to 1st dataframe per timestamp
    model_one_train[f'NDVI_{dates[i]}'] = ndvi 
    model_one_train[f'GNDVI_{dates[i]}'] = gndvi
    model_one_train[f'EVI_{dates[i]}'] = evi
    model_one_train[f'EVI2_{dates[i]}'] = evi2
    model_one_train[f'AVI_{dates[i]}'] = avi
    model_one_train[f'BSI_{dates[i]}'] = bsi
    model_one_train[f'SI_{dates[i]}'] = si    
    model_one_train[f'NDWI_{dates[i]}'] = ndwi
    model_one_train[f'NDMI_{dates[i]}'] = ndmi
    model_one_train[f'NPCRI_{dates[i]}'] = npcri


In [ ]:
# Add spectral indices statistics related to model_one_train dataframe
for i in spectral_indices:
    model_one_train[f'{i}_min'] = model_one_train.filter(regex = f'^{i}').min(axis = 1)
    model_one_train[f'{i}_max'] = model_one_train.filter(regex = f'^{i}').max(axis = 1)
    model_one_train[f'{i}_avg'] = model_one_train.filter(regex = f'^{i}').mean(axis = 1)
    model_one_train[f'{i}_std'] = model_one_train.filter(regex = f'^{i}').std(axis = 1)   

In [ ]:
# Add spectral indices statistics related to model_two_train dataframe
for i in spectral_indices:
    model_two_train[f'{i}_min'] = model_two_train.filter(regex = f'^{i}').min(axis = 1)
    model_two_train[f'{i}_max'] = model_two_train.filter(regex = f'^{i}').max(axis = 1)
    model_two_train[f'{i}_avg'] = model_two_train.filter(regex = f'^{i}').mean(axis = 1)
    model_two_train[f'{i}_std'] = model_two_train.filter(regex = f'^{i}').std(axis = 1)  

In [ ]:
pd.DataFrame(model_two_train)

In [ ]:
#Add the size of the fields

model_one_train['row_size'] = model_one_train.fid.map(row_data)
model_one_train['column_size'] = model_one_train.fid.map(column_data)
model_one_train['area'] = model_one_train.apply(lambda row: row.row_size * row.column_size, axis= 1)

model_one_train['pixels_data'] = model_one_train.fid.map(pixels_data)

# model_one_train = model_one_train.drop(columns = ['fid'])

In [ ]:
pd.DataFrame(model_one_train)

### Train and Test set
1. Determine which data to use for training and testing model.
2. Determine the best model to use to produce optimal results.


In [ ]:
# 1st dataframe
model_one_train_data = model_one_train[model_one_train.label != 0].copy()
model_one_test_data = model_one_train[model_one_train.label == 0].copy()

model_one_train_data= model_one_train_data.reset_index(drop = True)
model_one_test_data = model_one_test_data.reset_index(drop = True)

pd.DataFrame(model_one_train_data)

In [ ]:
# 2nd dataframe
model_two_train_data = model_two_train[model_one_train.label != 0].copy()
model_two_test_data = model_two_train[model_one_train.label == 0].copy()

model_two_train_data= model_two_train_data.reset_index(drop = True)
model_two_test_data = model_two_test_data.reset_index(drop = True)

pd.DataFrame(model_two_test_data)


In [ ]:
(model_one_train_data.shape, model_one_test_data.shape), (model_two_train_data.shape, model_two_test_data.shape)


In [ ]:
model_one_test_data.drop(columns='label',inplace = True, axis= 1)
model_two_test_data.drop(columns='label',inplace = True, axis= 1)


In [ ]:
pd.DataFrame(model_two_test_data)

In [ ]:
model_one_train_X = model_one_train_data.drop('label', axis = 1)
model_one_train_y = model_one_train_data.label

pd.DataFrame(model_one_train_y)

In [ ]:
#Model 2 
model_two_train_X = model_two_train_data.drop('label', axis = 1)
model_two_train_y = model_two_train_data.label

In [ ]:
model_one_train_X.shape, model_one_test_data.shape

# Step 3: Train

model used: catboost

#### Why Catboost
1. Reduces the need for hyper-parameter tuning as its default Parameters are good to work with.
2. Faster training and predictions

In [ ]:

import sklearn

from sklearn.utils.class_weight import compute_class_weight

from sklearn.inspection import permutation_importance
from scipy.stats.mstats import gmean


# Models
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import BaggingClassifier
import catboost
from catboost import CatBoostClassifier


import eli5
from eli5.sklearn import PermutationImportance


from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn.metrics import log_loss

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(model_one_train_X, model_one_train_y, test_size= 0.2,  stratify = model_one_train_y, random_state = 5 , shuffle = True)


In [ ]:

model_cb = CatBoostClassifier(iterations = 2000, random_state = 2021, task_type = "GPU")
model_cb.fit(X_train, y_train ,eval_set = (X_test, y_test), plot = True)

In [ ]:
from catboost import cv
from sklearn.metrics import accuracy_score
print('The test accuracy is :{:.6f}'.format(accuracy_score(y_test,model_cb.predict(X_test))))

In [ ]:
pi = PermutationImportance(model_cb, random_state = 2021, n_iter = 5)
pi.fit(X_test, y_test)

In [ ]:
eli5.show_weights(pi, feature_names = model_one_train_X.columns.tolist(), top = None)


### Cross Validation

1. Use CV to check if the model is not overfitting or underfitting
2. LDA will assist with reducing the number of features in training data to a more manageable number.
3. bagging classifier is used to reduced variance in datasets

In [ ]:
model_one_boost_one = CatBoostClassifier(n_estimators = 1500, learning_rate=0.03, depth = 6, random_state = 3000, bagging_temperature = 3, task_type = "GPU")
model_one_boost_two = CatBoostClassifier(n_estimators = 1100, learning_rate=0.02, depth = 6, random_state = 2000, bagging_temperature = 1, task_type = "GPU")


model_two_boost_one = CatBoostClassifier(n_estimators = 1500, learning_rate=0.03, depth = 6, random_state = 3000, bagging_temperature = 3, task_type = "GPU")
model_two_boost_two = CatBoostClassifier(n_estimators = 1100, learning_rate=0.02, depth = 6, random_state = 2000, bagging_temperature = 1, task_type = "GPU")

feature_reduction = LinearDiscriminantAnalysis()
reduced_variance = BaggingClassifier(base_estimator = feature_reduction, n_estimators = 30, random_state = 0)

#### Cross Validation config

##### Model one

In [ ]:
# Catboost
model_one_cv = cross_val_predict(model_one_boost_one, model_one_train_X, model_one_train_y, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Catboost with weights
model_one_cv_two = cross_val_predict(model_one_boost_two, model_one_train_X, model_one_train_y, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Catboost with weights
model_one_cv_three = cross_val_predict(reduced_variance, model_one_train_X, model_one_train_y, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Weighted Average of above 3 (in two steps)
weighted_average = (0.72 * model_one_cv) + ((1 - 0.72) * model_one_cv_two)

##### Model two

In [ ]:
# Catboost
model_two_cv = cross_val_predict(model_two_boost_one, model_two_train_X, model_two_train_y, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Catboost with weights
model_two_cv_two = cross_val_predict(model_two_boost_two, model_two_train_X, model_two_train_y, cv = 5, method = "predict_proba", verbose = 5)

In [ ]:
# Bagged LDA
model_two_cv_three = cross_val_predict(reduce_variance, model_two_train_X, model_two_train_y, cv = 5, method = "predict_proba", verbose = 5)

### Train and Predictions

Train the model using the cross validations configured.

##### model one

In [ ]:
model_one_boost_one.fit(model_one_train_X, model_one_train_y)

In [ ]:
model_one_boost_two.fit(model_one_train_X, model_one_train_y)

In [ ]:
bc.fit(model_one_train_X, model_two_train_y)

In [ ]:
test_preds_model_one = model_one_boost_one.predict_proba(model_one_test_data)
test_two_preds_model_one = model_one_boost_one.predict_proba(model_one_test_data)
test_three_preds_model_one  = reduced_variance.predict_proba(model_one_test_data)

In [ ]:

test_preds_all = (0.72 * test_preds_model_one) + ((1 - 0.72) * test_two_preds_model_one)
test_preds_all = (0.7 * test_preds_all) + ((1 - 0.7) * test_three_preds_model_one)

##### Model two

In [ ]:
cb_pixels.fit(model_two_train_X, model_two_train_y)

In [ ]:
cb2_pixels.fit(model_two_train_X, model_two_train_y)

In [ ]:
test_preds_pixels_1 = model_two_boost_one.predict_proba(model_two_test_data)
test_preds_pixels_2 = model_two_boost_two.predict_proba(model_two_test_data)

In [ ]:
# combine the two predictions
test_preds_pixels = (0.76 * test_preds_pixels_1) + ((1 - 0.76) * test_preds_pixels_2)

In [ ]:
# combine the two models to get final prediction
test_preds = (0.68 * test_preds_model_one) + ((1 - 0.68) * test_preds_pixels)

# Step 4: Submission file

In [ ]:
test_preds = pd.DataFrame(test_preds)

In [ ]:
sample_submission.Crop_ID_1 = test_preds[0]
sample_submission.Crop_ID_2 = test_preds[1]
sample_submission.Crop_ID_3 = test_preds[2]
sample_submission.Crop_ID_4 = test_preds[3]
sample_submission.Crop_ID_5 = test_preds[4]
sample_submission.Crop_ID_6 = test_preds[5]
sample_submission.Crop_ID_7 = test_preds[6]

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv("SampleSub.csv", index = False)